In [15]:
import tensorflow as tf
import numpy as np
from PIL import Image

# --- Đường dẫn ---
model_path = r"plant_model.tflite"
labels_path = r"labels.txt"
img_path = r"D:\plant_villages\color\Corn_(maize)___Common_rust\RS_Rust 2738.JPG"

# --- Load model ---
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# --- Load labels ---
with open(labels_path, "r", encoding="utf-8") as f:
    labels = [line.strip() for line in f.readlines()]

# --- Xử lý ảnh đúng cách (chuẩn hóa [0,1]) ---
img = Image.open(img_path).convert("RGB").resize((224, 224))
input_data = np.expand_dims(np.float32(img) / 255.0, axis=0)

# --- Dự đoán ---
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])[0]

# --- Hiển thị kết quả ---
pred_idx = np.argmax(output_data)
confidence = output_data[pred_idx]
print(f"👉 Nhãn: {labels[pred_idx]}")
print(f"🔹 Độ tin cậy: {confidence:.3f}")

# Top 3 kết quả
top3_idx = np.argsort(output_data)[::-1][:3]
print("\n🔝 Top 3 dự đoán:")
for i in top3_idx:
    print(f"{labels[i]}: {output_data[i]:.3f}")


👉 Nhãn: Corn_(maize)___healthy
🔹 Độ tin cậy: 0.939

🔝 Top 3 dự đoán:
Corn_(maize)___healthy: 0.939
Grape___healthy: 0.060
Corn_(maize)___Common_rust: 0.001


In [17]:
from tensorflow.keras.models import load_model
from PIL import Image
import numpy as np

model = load_model(r"plant_model_final.h5")

def predict_image(img_path):
    img = Image.open(img_path).convert("RGB").resize((224, 224))
    arr = np.expand_dims(np.float32(img) / 255.0, axis=0)
    pred = model.predict(arr)
    idx = np.argmax(pred)
    conf = np.max(pred)
    print(f"Ảnh: {img_path}")
    print(f"--> Nhãn: {idx}, Độ tin cậy: {conf:.4f}")

# test vài ảnh
predict_image(r"D:\plant_villages\color\Corn_(maize)___Common_rust\RS_Rust 2738.JPG")
predict_image(r"D:\plant_villages\color\Apple___Apple_scab\ff99efdc-a9f8-4360-9c64-f8274f456be5___FREC_Scab 3161.JPG")
predict_image(r"C:\Users\impor\Downloads\3978901481_3be68f63c3_z.jpg")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 709ms/step
Ảnh: D:\plant_villages\color\Corn_(maize)___Common_rust\RS_Rust 2738.JPG
--> Nhãn: 5, Độ tin cậy: 0.9390
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
Ảnh: D:\plant_villages\color\Apple___Apple_scab\ff99efdc-a9f8-4360-9c64-f8274f456be5___FREC_Scab 3161.JPG
--> Nhãn: 5, Độ tin cậy: 0.9369
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Ảnh: C:\Users\impor\Downloads\3978901481_3be68f63c3_z.jpg
--> Nhãn: 5, Độ tin cậy: 0.9365
